In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

In [ ]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data", #data 다운
    train=True,  #train data 다운
    download=True, # True면 다운로드, False면 다운도르 x
    transform=ToTensor(), # 원래는 = 0~255값인데, 0~1사이 값으로 정규화 변환시키는 것 
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)
# DataLodaer에 대해서 공부하기

# 5만장이 train, 1만이 test

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


# 새 섹션

In [ ]:
# traindata를 한번에 연산을 못해 메모리가 딸림 그래서 쪼개씩 받아가는거야
batch_size = 64 # 학습해 나갈 양? 

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size) #data, target 두 값을 가짐
# train data를 batch_size로 쪼개주는거? 공부해보기
test_dataloader = DataLoader(test_data, batch_size =batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break
# N = 64개, C = 1 (흑백만), 28*28 사이즈

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu" # gpu 쓸수 있는 환경이면 gpu쓰겠다.

print("Using {} device".format(device))

# Define model
class NeuralNetwork(nn.Module):# nn.Module을 상속받는다.
    def __init__(self): # init는 무조건 실행되는 것
        super(NeuralNetwork, self).__init__() # super().__init__() 도 가능
        # super()로 기반 클래스(부모 클래스)를 초기화해줌으로써, 기반 클래스의 속성을 subclass가 받아오도록 한다.
        self.flatten = nn.Flatten() # 이미지를 1행열로 바꿔주는 함수
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512), # 28*28 - > 512로가는 linear 행렬
            # 이때 w가 작용됨 이게 학습되는 parameter
            nn.ReLU(),# activation function
            nn.Linear(512, 512), 512 -> 512
            nn.ReLU(),
            nn.Linear(512, 10) #클래스 갯수 10개
        )
#   forward() 함수는 model 객체를 데이터와 함께 호출하면 자동으로 실행이됩니다
    def forward(self, x): #x를  flatten 해주고 Sequential 모델에 넣어주는 식
        x = self.flatten(x) # 64*784
        logits = self.linear_relu_stack(x)
        return logits #softmax의 인풋을 logits라고 많이 불림

# to.(device) # gpu로 보낼지 cpu로 보낼지 하는겨
model = NeuralNetwork().to(device)
# model2 = NeuralNetwork().to(device) 같은 구조에 다른 모델도 만들 수있음
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
#SGD(Stochastic gradient descent) 경사하강법
# nn.Module에 parameters에 있는 거를 수정하겠다.


In [ ]:
def Train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train() # 우리가 만든 Class안에 상속받은거에서 train 실행, 
    for batch, (X, y) in enumerate(dataloader): # enumerate는 인덱스랑 같이 값을 저장
       # 64개씩 전체를 보겠다. batch는 몇번쨰인지
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X) 
        loss = loss_fn(pred, y) #예측값이랑 비교, sottmax값을 안넣어주는것은
                                # 수치적인 안정성 때문에? (나중에 배운대)
        # Backpropagation
        optimizer.zero_grad() # 미분값 초기화하는거임 반복할때마다 저장되거든
        loss.backward() #loss의 미분해주는거
        optimizer.step() # parmeter update

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X) # loss.item으로 loss에 저장된 값 가져올 수 있음 
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()# eval함수 써라
    test_loss, correct = 0, 0
    with torch.no_grad(): # 미분값 구하지 않겠다.
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 5 # 총 5번 반복해서 쓰겠다.
# 전체dataset을 다하는것을  1 epochs
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.299098  [    0/60000]
loss: 2.290434  [ 6400/60000]
loss: 2.279569  [12800/60000]
loss: 2.277494  [19200/60000]
loss: 2.253355  [25600/60000]
loss: 2.230724  [32000/60000]
loss: 2.238293  [38400/60000]
loss: 2.206736  [44800/60000]
loss: 2.207496  [51200/60000]
loss: 2.184676  [57600/60000]
Test Error: 
 Accuracy: 51.8%, Avg loss: 2.174189 

Epoch 2
-------------------------------
loss: 2.176063  [    0/60000]
loss: 2.170564  [ 6400/60000]
loss: 2.122715  [12800/60000]
loss: 2.142454  [19200/60000]
loss: 2.096408  [25600/60000]
loss: 2.045932  [32000/60000]
loss: 2.073854  [38400/60000]
loss: 1.998848  [44800/60000]
loss: 2.007031  [51200/60000]
loss: 1.952778  [57600/60000]
Test Error: 
 Accuracy: 59.6%, Avg loss: 1.939935 

Epoch 3
-------------------------------
loss: 1.961243  [    0/60000]
loss: 1.941902  [ 6400/60000]
loss: 1.832332  [12800/60000]
loss: 1.871679  [19200/60000]
loss: 1.779969  [25600/60000]
loss: 1.723692  [32000/600